In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pulses import RadarPulseGenerator, CommPulseGenerator
import torch
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)
t = torch.linspace(0, 1e-4, int(1e5), device=device)
dt = t[1] - t[0]
# Define parameters
T = 1e-4       # Total duration of radar pulse (1e-4 seconds)
B = 1e6        # Bandwidth of radar pulse (1 MHz)
K = 5         # Number of users (OFDMA users)
L = 100        # Number of bits in radar pulse duration
Tb = T / L     # Duration of each bit interval
distance = 50
my_tau = 2*distance / 3e8
N0 = 10
A = 0.8

myCommPulses = CommPulseGenerator(t, Tb, T, device)
myRadar = RadarPulseGenerator(t, T=T, device=device)

radar_pulse = myRadar.LFM_pulse(B)
print(torch.sum(torch.abs(radar_pulse)**2)*dt)
# radar_pulse = radar_pulse / torch.sqrt(torch.sum(torch.abs(radar_pulse)**2)*dt)

dis, ta = myRadar.montecarlo_estimation(radar_pulse, N0, 2040, np.sqrt(A), 3e8, 10000)
print(f'Mean value->{torch.mean(dis)}, \t Variance->{torch.var(dis)}')
print(f'Mean value->{torch.mean(ta)}, \t Variance->{torch.var(ta)}')

cuda
tensor(0.0001, device='cuda:0')


100%|██████████| 10000/10000 [00:22<00:00, 447.09it/s]


Mean value->2039.875, 	 Variance->0.5375714898109436
Mean value->1.3599165868072305e-05, 	 Variance->2.3892016806463184e-17


100%|██████████| 10000/10000 [00:23<00:00, 428.88it/s]

Mean value->2039.8731689453125, 	 Variance->0.1357789784669876
Mean value->1.3599154044641182e-05, 	 Variance->6.034721387889296e-18


In [ ]:
3 / 2 / np.pi**2 / B**2 / T * N0 * dt / A**2
2039.8763427734375


3.9590919399212985